Этот ноутбук предназначен для сбора корпуса финских текстов из разных источников:
- новостные сайты (Helsingin Sanomat, KU.fi, Heili и др.)
- адаптированные книги на финском языке (PDF, DOCX)

Цель: собрать разнообразные аутентичные и учебные тексты для дальнейшей обработки и генерации упражнений.
В итоге целью было получить csv файл с текстами разных тематик и длины.


# Сбор корпуса их новостных сайтов

Почему было принято решение собирать именно тексты новостных сайтов?

**Актуальный язык**:
Новости отражают современный финский язык — лексику, фразеологию и грамматику, которые реально используются в обществе. При этом сочетается как формальный, так и разговорный стиль.


**Разнообразие тем**:
В новостях охватываются:
политика, экология, технологии, медицина, культура и т.д.
Это важно для обучения лексике по темам и тематического моделирования (LDA).

**Формальный стиль**:
Новостной язык чёткий, структурированный и часто грамматически корректен — хорошая основа для создания упражнений.

**Большой объём и доступность**:
Новости публикуются каждый день и легкодоступны для парсинга.

In [23]:
#импорт библиотек
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from tqdm import tqdm
import re

1. Helsingin Sanomat

In [2]:
#Указываем заголовки для запроса (чтобы сайт не заблокировал), и список рубрик для парсинга. Устанавливаем ограничения на количество статей (по факту, оно не пригодилось)
#рубрики были отобраны вручную, некоторые были также удалены из-за большого количества платного контента
HEADERS = {"User-Agent": "Mozilla/5.0"}
MAX_PER_RUBRIC = 150
MAX_TOTAL = 1000

rubrics = {
    "maailma": "https://www.hs.fi/maailma/",
    "paakirjoitukset": "https://www.hs.fi/paakirjoitukset/",
    "alueet": "https://www.hs.fi/alueet/",
    "suomi": "https://www.hs.fi/suomi/",
    "politiikka": "https://www.hs.fi/politiikka/",
    "helsinki": "https://www.hs.fi/helsinki/",
    "mielipide": "https://www.hs.fi/mielipide/",
}

#функция ищет все ссылки на статьи из заданной рубрики
def get_article_links(rubric_url, rubric_name, max_links=MAX_PER_RUBRIC):
    article_links = set()
    response = requests.get(rubric_url, headers=HEADERS, timeout=10)
    soup = BeautifulSoup(response.content, "html.parser")

    rubric_path = "/" + rubric_name + "/"

    for a in soup.find_all("a", href=True):
        href = a["href"]
        if rubric_path in href and "/art-" in href:
            full_url = "https://www.hs.fi" + href.split("?")[0] if href.startswith("/") else href
            article_links.add(full_url)
        if len(article_links) >= max_links:
            break
    return list(article_links)

#функция извлекает текст статьи с конкретной страницы
def get_article_text_hs(url):
    r = requests.get(url, headers=HEADERS, timeout=10)
    soup = BeautifulSoup(r.content, "html.parser")
    article = soup.find("article")
    if not article:
        return None

    paragraphs = article.find_all("p")
    text = "\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))
    return text if len(text.split()) > 50 else None

#собираю все статьи: прохожусь по каждой рубрике, собираю ссылки и тексты, сохраняю их в список
total_saved = 0
articles = []

for rubric_name, rubric_url in rubrics.items():
    print(f"Рубрика: {rubric_name}")
    links = get_article_links(rubric_url, rubric_name, max_links=MAX_PER_RUBRIC)
    saved = 0
    i = 0
    while saved < MAX_PER_RUBRIC and i < len(links) and total_saved < MAX_TOTAL:
        url = links[i]
        text = get_article_text_hs(url)
        #на всякий случай делаю для себя вывод, что выводится именно тело статьи, а не заголовок или еще что-то.
        if saved == 1:
            print("\nПредпросмотр:")
            print("=" * 80)
            print(text[:1500])
            print("=" * 80)

        if text and len(text.split()) > 100:
            articles.append({
                "rubric": rubric_name,
                "url": url,
                "text": text
            })
            saved += 1
            total_saved += 1
            print(f"[{rubric_name}] {saved}: {url}")
        i += 1
        time.sleep(1.5)

    print(f"Сохранено {saved} статей из '{rubric_name}'\n")

#сохраняю в csv
df = pd.DataFrame(articles)
df.to_csv("hs_articles.csv", index=False, encoding="utf-8")
print(f"Всего сохранено: {total_saved} статей в hs_articles.csv")

Рубрика: maailma
[maailma] 1: https://www.hs.fi/maailma/art-2000011350682.html

Предпросмотр:
Venäjän hyökkäys|Ukrainalaisten arvioiden mukaan jopa 80 prosenttia venäläisten lennokkien kriittisestä elektroniikasta on lähtöisin Kiinasta.
Kiinan presidentti Xi Jinping ja Venäjän presidentti Vladimir Putin tapasivat toukokuussa Moskovassa.Kuva:Kirill Kudryavtsev / AFP
Kiinan ulkoministeri Wang Yi kertoi, ettei Kiina voi hyväksyä Venäjän häviötä Ukrainan sodassa.
Kiina tukee Venäjän sotateollisuutta toimittamalla elektroniikkaa ja muita tuotteita pakotteista huolimatta.
Venäjällä ollaan kuitenkin huolissaan maan kasvavasta riippuvuudesta Kiinaan.
KiinanulkoministeriWang Yikertoi torstaina, että Kiina ei voi hyväksyä tilannetta, jossa Venäjä häviäisi hyökkäyssotansa Ukrainassa.
Ulkoministeri kertoi asiasta EU:n ulkosuhteiden- ja turvallisuuspolitiikan korkealle edustajalleKaja Kallaksellediplomaattien välisessä tapaamisessa.
Asiasta kertoiuutiskanava CNNviitaten paikalla olleeseen viranomai

In [3]:
#в текст статей попали и названия, сразу очищаю
#загружаю файл
df = pd.read_csv("hs_articles.csv")

#очищаю третью колонкку, удаляю все до первой черты и саму черту
df.iloc[:, 2] = df.iloc[:, 2].str.replace(r'^.*?\|\s*', '', regex=True)

#сохраняю очищенный файл
df.to_csv("hs_articles_cleaned.csv", index=False)

#остальную предобработку буду делать уже когда будет собран полный корпус.

Далее код по сути аналогичен для других новостных сайтов.

2. KU.fi

In [4]:
MAX_ARTICLES = 50
CSV_PATH = "ku_articles.csv"

def parse_ku_article(url):
    res = requests.get(url, headers=HEADERS, timeout=10)
    soup = BeautifulSoup(res.content, "html.parser")

    content = soup.find("div", class_="entry-content")
    if not content:
        print(f"Не найден блок в {url}")
        return None

    paragraphs = content.find_all("p")
    text = "\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))
    return text

def collect_ku_articles(max_articles=MAX_ARTICLES):
    base_url = "https://www.ku.fi"
    visited = set()
    collected = 0
    records = []

    res = requests.get(base_url, headers=HEADERS, timeout=10)
    soup = BeautifulSoup(res.content, "html.parser")

    links = [a["href"] for a in soup.find_all("a", href=True) if "/artikkeli/" in a["href"]]

    for link in links:
        if collected >= max_articles:
            break
        url = base_url + link if link.startswith("/") else link
        if url in visited:
            continue

        text = parse_ku_article(url)
        if text and len(text) > 200:
            records.append({
                "source": "ku",
                "url": url,
                "text": text
            })
            print(f"Сохранено {collected+1}: {url}")
            collected += 1

        visited.add(url)
        time.sleep(1)

    df = pd.DataFrame(records)
    df.to_csv(CSV_PATH, index=False, encoding="utf-8")
    print(f"Сохранено {collected} статей в {CSV_PATH}")

collect_ku_articles()

Сохранено 1: https://www.ku.fi/artikkeli/5185343-javier-milein-sota-argentiinan-mediaa-vastaan-emme-vihaa-toimittajia-tarpeeksi
Сохранено 2: https://www.ku.fi/artikkeli/5182220-ku-listasi-suomen-vaikuttavimmat-vasemmistolaiset-katso-10-nimea
Сохранено 3: https://www.ku.fi/artikkeli/5181978-noora-kotilaisen-kolumni-tappaminen-sodassa-on-juridisesti-oikeutettua-mutta-lopulta-yksilo-on-yksin-tekojensa-kanssa
Сохранено 4: https://www.ku.fi/artikkeli/5181613-kaannekohtien-kevat-mita-jai-mieleen-politiikan-alkuvuodesta-2025
Сохранено 5: https://www.ku.fi/artikkeli/5181174-nelja-nostoa-sotilasliiton-huippukokouksesta-nato-suostui-trumpin-saneluun
Сохранено 6: https://www.ku.fi/artikkeli/5185886-mathenge-on-kenian-villi-lupiini-ja-jattipalsami-nyt-pakolaisleirin-tytot-ovat-valjastaneet-haitallisen-vieraslajin-hyotykayttoon
Сохранено 7: https://www.ku.fi/artikkeli/5184944-laivayhteys-helsingista-tallinnaan-tayttaa-60-vuotta-jarisytti-naapurisuhteita
Сохранено 8: https://www.ku.fi/artikkeli/5184

3.  Hämeenlinnakaupunkiuutiset

In [7]:
MAX_ARTICLES = 50
CSV_PATH = "hameenlinna_articles.csv"

def parse_article(url):
    res = requests.get(url, headers=HEADERS, timeout=10)
    soup = BeautifulSoup(res.content, "html.parser")

    content = soup.find("div", class_="diks-article__inner-content")
    if not content:
        return None

    paragraphs = content.find_all("p")
    text = "\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))
    return text if len(text.split()) >= 50 else None

def collect_articles(max_articles=MAX_ARTICLES):
    base_url = "https://www.hameenlinnankaupunkiuutiset.fi"
    res = requests.get(base_url, headers=HEADERS, timeout=10)
    soup = BeautifulSoup(res.content, "html.parser")

    links = [a["href"] for a in soup.find_all("a", href=True)
             if "/paikalliset/" in a["href"] and any(char.isdigit() for char in a["href"].split("/")[-1])]
    links = list(dict.fromkeys(links))

    records = []
    visited = set()

    for link in links:
        if len(records) >= max_articles:
            break
        url = base_url + link if link.startswith("/") else link
        if url in visited:
            continue

        text = parse_article(url)
        if text:
            records.append({
                "source": "hameenlinna",
                "url": url,
                "text": text
            })
            print(f"Сохранено {len(records)}: {url}")

        visited.add(url)
        time.sleep(1)

    pd.DataFrame(records).to_csv(CSV_PATH, index=False, encoding="utf-8")
    print(f"Сохранено {len(records)} статей в {CSV_PATH}")

collect_articles()

Сохранено 1: https://www.hameenlinnankaupunkiuutiset.fi/paikalliset/8629711
Сохранено 2: https://www.hameenlinnankaupunkiuutiset.fi/paikalliset/8629583
Сохранено 3: https://www.hameenlinnankaupunkiuutiset.fi/paikalliset/8603797
Сохранено 4: https://www.hameenlinnankaupunkiuutiset.fi/paikalliset/8629848
Сохранено 5: https://www.hameenlinnankaupunkiuutiset.fi/paikalliset/8629918
Сохранено 6: https://www.hameenlinnankaupunkiuutiset.fi/paikalliset/8609968
Сохранено 7: https://www.hameenlinnankaupunkiuutiset.fi/paikalliset/8609951
Сохранено 8: https://www.hameenlinnankaupunkiuutiset.fi/paikalliset/8593582
Сохранено 9: https://www.hameenlinnankaupunkiuutiset.fi/paikalliset/8629903
Сохранено 10: https://www.hameenlinnankaupunkiuutiset.fi/paikalliset/8629891
Сохранено 11: https://www.hameenlinnankaupunkiuutiset.fi/paikalliset/8615169
Сохранено 12: https://www.hameenlinnankaupunkiuutiset.fi/paikalliset/8614752
Сохранено 13: https://www.hameenlinnankaupunkiuutiset.fi/paikalliset/8610222
Сохранен

In [11]:
df = pd.read_csv("hameenlinna_articles.csv")

#удаляю фразу "Lue lisää aiheesta" из колонки с текстом
df["text"] = df["text"].str.replace("Lue lisää aiheesta", "", regex=False)

#сохраняю
df.to_csv("hameenlinna_articles_cleaned.csv", index=False)

4. Heili.fi

In [10]:
MAX_ARTICLES = 50
CSV_PATH = "heili_articles.csv"

def parse_heili_article(url):
    res = requests.get(url, headers=HEADERS, timeout=10)
    soup = BeautifulSoup(res.content, "html.parser")

    content = soup.find("div", class_="diks-article__inner-content")
    if not content:
        print(f"Не найден блок в {url}")
        return None

    paragraphs = content.find_all("p")
    text = "\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))
    return text if len(text.split()) >= 50 else None

def collect_heili_articles(max_articles=MAX_ARTICLES):
    base_url = "https://www.heili.fi"
    visited = set()
    collected = 0
    records = []

    res = requests.get(base_url, headers=HEADERS, timeout=10)
    soup = BeautifulSoup(res.content, "html.parser")

    links = [a["href"] for a in soup.find_all("a", href=True)
             if "/paikalliset/" in a["href"] and any(char.isdigit() for char in a["href"].split("/")[-1])]

    for link in links:
        if collected >= max_articles:
            break
        url = base_url + link if link.startswith("/") else link
        if url in visited:
            continue

        text = parse_heili_article(url)
        if text:
            records.append({
                "source": "heili",
                "url": url,
                "text": text
            })
            print(f"Сохранено {collected + 1}: {url}")
            collected += 1

        visited.add(url)
        time.sleep(1)

    df = pd.DataFrame(records)
    df.to_csv(CSV_PATH, index=False, encoding="utf-8")
    print(f"Сохранено {collected} статей в {CSV_PATH}")

collect_heili_articles()

Сохранено 1: https://www.heili.fi/paikalliset/8600007
Сохранено 2: https://www.heili.fi/paikalliset/8580276
Сохранено 3: https://www.heili.fi/paikalliset/8600434
Сохранено 4: https://www.heili.fi/paikalliset/8654118
Сохранено 5: https://www.heili.fi/paikalliset/8645334
Сохранено 6: https://www.heili.fi/paikalliset/8645303
Сохранено 7: https://www.heili.fi/paikalliset/8643125
Сохранено 8: https://www.heili.fi/paikalliset/8580194
Сохранено 9: https://www.heili.fi/paikalliset/8630242
Сохранено 10: https://www.heili.fi/paikalliset/8632992
Сохранено 11: https://www.heili.fi/paikalliset/8632929
Сохранено 12: https://www.heili.fi/paikalliset/8629659
Сохранено 13: https://www.heili.fi/paikalliset/8627573
Сохранено 14: https://www.heili.fi/paikalliset/8619079
Сохранено 15: https://www.heili.fi/paikalliset/8627581
Сохранено 16: https://www.heili.fi/paikalliset/8627393
Сохранено 17: https://www.heili.fi/paikalliset/8584425
Сохранено 18: https://www.heili.fi/paikalliset/8620722
Сохранено 19: https

In [12]:
df = pd.read_csv("heili_articles.csv")

#удаляю фразу "Lue lisää aiheesta" из колонки с текстом
df["text"] = df["text"].str.replace("Lue lisää aiheesta", "", regex=False)

#сохраняю
df.to_csv("heili_articles_cleaned.csv", index=False)

5. Karkkilalainen

In [13]:
MAX_ARTICLES = 50
CSV_PATH = "karkkilalainen_articles.csv"

def parse_karkkilalainen_article(url):
    res = requests.get(url, headers=HEADERS, timeout=10)
    soup = BeautifulSoup(res.content, "html.parser")

    content = soup.find("div", class_="diks-article__inner-content")
    if not content:
        print(f"Не найден блок в {url}")
        return None

    paragraphs = content.find_all("p")
    text = "\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))
    return text

def collect_karkkilalainen_articles(max_articles=MAX_ARTICLES):
    base_url = "https://www.karkkilalainen.fi"
    visited = set()
    collected = 0
    records = []

    res = requests.get(base_url, headers=HEADERS, timeout=10)
    soup = BeautifulSoup(res.content, "html.parser")

    links = [a["href"] for a in soup.find_all("a", href=True)
             if "/paikalliset/" in a["href"] and any(char.isdigit() for char in a["href"].split("/")[-1])]

    for link in links:
        if collected >= max_articles:
            break
        url = base_url + link if link.startswith("/") else link
        if url in visited:
            continue

        text = parse_karkkilalainen_article(url)
        if text and len(text) > 200:
            records.append({
                "source": "karkkilalainen",
                "url": url,
                "text": text
            })
            print(f"Сохранено {collected + 1}: {url}")
            collected += 1

        visited.add(url)
        time.sleep(1)

    df = pd.DataFrame(records)
    df.to_csv(CSV_PATH, index=False, encoding="utf-8")
    print(f"Сохранено {collected} статей в {CSV_PATH}")

collect_karkkilalainen_articles()

Сохранено 1: https://www.karkkilalainen.fi/paikalliset/8568785
Сохранено 2: https://www.karkkilalainen.fi/paikalliset/8673870
Сохранено 3: https://www.karkkilalainen.fi/paikalliset/8674497
Сохранено 4: https://www.karkkilalainen.fi/paikalliset/8670609
Сохранено 5: https://www.karkkilalainen.fi/paikalliset/8669579
Сохранено 6: https://www.karkkilalainen.fi/paikalliset/8668574
Сохранено 7: https://www.karkkilalainen.fi/paikalliset/8668265
Сохранено 8: https://www.karkkilalainen.fi/paikalliset/8659577
Сохранено 9: https://www.karkkilalainen.fi/paikalliset/8668290
Сохранено 10: https://www.karkkilalainen.fi/paikalliset/8667704
Сохранено 11: https://www.karkkilalainen.fi/paikalliset/8664870
Сохранено 12: https://www.karkkilalainen.fi/paikalliset/8658445
Сохранено 13: https://www.karkkilalainen.fi/paikalliset/8656327
Сохранено 14: https://www.karkkilalainen.fi/paikalliset/8655173
Сохранено 15: https://www.karkkilalainen.fi/paikalliset/8658698
Сохранено 16: https://www.karkkilalainen.fi/paika

In [48]:
df = pd.read_csv("karkkilalainen_articles.csv")

#удаляю фразу "Lue lisää aiheesta" из колонки с текстом
df["text"] = df["text"].str.replace("Lue lisää aiheesta", "", regex=False)

#сохраняю
df.to_csv("karkkilalainen_articles_cleaned.csv", index=False)

6. Pkank

In [17]:
MAX_ARTICLES = 50
CSV_PATH = "pkank_articles.csv"

def parse_pkank_article(url):
    res = requests.get(url, headers=HEADERS, timeout=10)
    soup = BeautifulSoup(res.content, "html.parser")

    content = soup.find("div", class_="ank-text")
    if not content:
        print(f"Не найден блок в {url}")
        return None

    paragraphs = content.find_all("p", class_="teksti")
    text = "\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))
    return text if len(text) > 200 else None

def collect_pkank_articles(max_articles=MAX_ARTICLES):
    base_url = "https://www.pkank.fi"
    visited = set()
    collected = 0
    records = []

    res = requests.get(base_url, headers=HEADERS, timeout=10)
    soup = BeautifulSoup(res.content, "html.parser")

    links = [a["href"] for a in soup.find_all("a", href=True)
             if "/uutiset/" in a["href"] and any(char.isdigit() for char in a["href"].split("/")[-1])]

    for link in links:
        if collected >= max_articles:
            break
        url = base_url + link if link.startswith("/") else link
        if url in visited:
            continue

        text = parse_pkank_article(url)
        if text:
            records.append({
                "source": "pkank",
                "url": url,
                "text": text
            })
            print(f"Сохранено {collected+1}: {url}")
            collected += 1

        visited.add(url)
        time.sleep(1)

    df = pd.DataFrame(records)
    df.to_csv(CSV_PATH, index=False, encoding="utf-8")
    print(f"\nСохранено {collected} статей в {CSV_PATH}")

collect_pkank_articles()

Сохранено 1: https://www.pkank.fi/uutiset/anjalassa-ja-voikkaalla-torjutaan-vieraskasveja-talkoilla-6.19.36938.62aa160531
Сохранено 2: https://www.pkank.fi/uutiset/wiipurintien-markkinat-vie-jalleen-keskiajan-tunnelmiin-6.19.36934.7c8ceec77e
Сохранено 3: https://www.pkank.fi/uutiset/tyo-kymenlaakson-luonnon-eteen-toi-ymparistopalkinnon-6.19.36823.498d07553b
Сохранено 4: https://www.pkank.fi/uutiset/kiinteistoilla-elinvoimaa-kotkaan-6.19.35521.a4e3849cab
Сохранено 5: https://www.pkank.fi/uutiset/korjausurakka-sulkee-alikulkukaytavan-viikoksi-kouvolan-lehtomaessa-6.19.36935.6107a5f4b3
Сохранено 6: https://www.pkank.fi/uutiset/kotkalaislahtoinen-that-band-from-finland-voitti-suomen-paras-kantribiisi-kilpailun-6.19.36939.9290fff906
Сохранено 7: https://www.pkank.fi/uutiset/mielipide-suomi-tarvitsee-positiivista-maahanmuuttoa-ei-rasismia-6.19.36931.6d1d994423
Сохранено 8: https://www.pkank.fi/uutiset/mika-on-kouvolan-keskustateko-2025--aanestys-on-kaynnissa-6.19.36936.0f7d61e2d3

Сохранено 

На данный момент у нас есть 6 отдельных csv файлов для каждой газеты - объединяем их вместе.

In [49]:
file_paths = [
    "heili_articles_cleaned.csv",
    "hameenlinna_articles_cleaned.csv",
    "karkkilalainen_articles_cleaned.csv",
    "ku_articles.csv",
    "hs_articles_cleaned.csv",
    "pkank_articles.csv"
]

dfs = [pd.read_csv(path) for path in file_paths]
for df, path in zip(dfs, file_paths):
    df["source_file"] = path

combined_df = pd.concat(dfs, ignore_index=True)
combined_df.to_csv("all_articles_combined.csv", index=False, encoding="utf-8")

print(f"Объединено")

Объединено


# Сбор книжного корпуса

Помимо новостных текстов, я также включаю в корпус материалы из учебной и художественной литературы. Это позволяет расширить языковое разнообразие и сделать корпус полезным для генерации заданий с разными уровнями сложности и жанрами текста.



*  Источниками послужили:

PDF-файлы с адаптированными книгами на финском языке (короткие, детские рассказы)

Bilingual-файлы (финско-русские), из которых извлекаются только финские реплики

Все тексты проходят фильтрацию по языку: мы исключаем строки с русскими символами, чтобы сохранить только финскую часть текста.

Художественные тексты намеренно не использовались в этом этапе, поскольку они часто содержат устаревшую или нестандартную лексику, а также неестественные для повседневного языка конструкции.

In [19]:
#установка библиотек для работы с пдф, ворд файлами, os для работы с файлами
!pip install pymupdf
!pip install python-docx
import fitz
from docx import Document
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 3.2 MB/s eta 0:00:00


Открываю пдф и извлекаю текст со всех страниц (вручную удаляю первую ненужную страницу)

In [20]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page_num, page in enumerate(doc):
            if page_num == 0:
                continue  # пропускаем первую страницу, т.к. на ней просто содержание
            text += page.get_text()
    return text

Тоже самое для другой книги, но еще удаляем все кириллические символы, т.к. книга сразу с переводом

In [21]:
def extract_finnish_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""

    for page_num in range(0, len(doc)):
        page = doc.load_page(page_num)
        page_text = page.get_text()
        for line in page_text.splitlines():
            line = line.strip()
            if line and not re.search(r'[а-яА-ЯёЁ]', line):
                text += line + "\n"

    return text

Обработка ворд-файла

In [22]:
def extract_finnish_from_bilingual(docx_path):
    doc = Document(docx_path)
    text = ""
    for para in doc.paragraphs:
        line = para.text.strip()
        if line and not re.search(r'[а-яА-ЯёЁ]', line):  # исключаем строки с кириллицей
            text += line + "\n"
    return text

Объединяю все в один файл, разделяя при этом на чанки (не более 10 000 слов), чтобы в дальнейшем было легче обрабатывать данные

In [35]:
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 26.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [40]:
import os
import stanza

#скачиваю stanza для финского языка, чтобы разбить сплошной текст на предложения
stanza.download("fi")
#нужна только токенизация
nlp = stanza.Pipeline("fi", processors="tokenize", tokenize_no_ssplit=False)

#функция для разделения текста на предложения
def split_into_sentences(text):
    doc = nlp(text)
    return [sent.text for sent in doc.sentences]

#функция для разделения текста на чанки
def save_texts_sentence_chunks(output_dir, *texts, chunk_size=10000):
    #создаю папку для вывода частей текстов
    os.makedirs(output_dir, exist_ok=True)
    #список для хранения всех предложений
    all_sentences = []

    #прохожу по всем текстам, разбиваю каждый текст на предложения и добавляю их в список
    for text in texts:
        all_sentences.extend(split_into_sentences(text))

    #список для хранения чанков
    chunks = []
    #временный список для текущего чанка
    current_chunk = []
    #счетчик слов для текущего чанка
    current_word_count = 0
    #счетчик чанков, для того, чтобы потом назвать чанки с правильнйо нумерацией
    chunk_counter = 1

    #прохожу по предложениям, считаю слова в предложении
    for sent in all_sentences:
        sent_word_count = len(sent.split())
        #если добавление предложения ведет к превышению размера чанка, то формируем текст текущей части, сохраняем в ДатаФрейм и записываем в файл
        if current_word_count + sent_word_count > chunk_size:
            chunk_text = " ".join(current_chunk)
            df = pd.DataFrame({"text": [chunk_text]})
            filename = os.path.join(output_dir, f"text_chunk_{chunk_counter}.csv")
            df.to_csv(filename, index=False)
            print(f"Сохранён файл: {filename}, слов: {current_word_count}")
            #увеличиваю счетчик чанков и сбрастываю временный список для текущего чанка и количество слов в текущем чанке
            chunk_counter += 1
            current_chunk = []
            current_word_count = 0
        #добавляю предложение в текущую часть (чанк)
        current_chunk.append(sent)
        #увеличиваю количество слов в текущей части (чанке)
        current_word_count += sent_word_count

    #если после всего в конце останется неполная часть, то ее тоже сохраняем
    if current_chunk:
        chunk_text = " ".join(current_chunk)
        df = pd.DataFrame({"text": [chunk_text]})
        filename = os.path.join(output_dir, f"text_chunk_{chunk_counter}.csv")
        df.to_csv(filename, index=False)
        print(f"Сохранён файл: {filename}, слов: {current_word_count}")

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: fi (Finnish) ...
INFO:stanza:File exists: /root/stanza_resources/fi/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: fi (Finnish):
| Processor | Package |
-----------------------
| tokenize  | tdt     |
| mwt       | tdt     |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Done loading processors!


Собственно, запускаем все наши функции в основном блоке

In [42]:
if __name__ == "__main__":
    pdf_text = extract_text_from_pdf("Sbornik-knig-na-uproshchyonnom-finskom-yazyke-selkokieli-Finnish-_RuLit_Me.pdf")
    muumi_text = extract_finnish_from_pdf("Jansson_Muumipeikko_bilingua.pdf")
    finrus_text = extract_finnish_from_bilingual("Финские тексты.docx")
    adapted_text = extract_finnish_from_bilingual("адаптированные худ. тексты.docx")

    save_texts_sentence_chunks("output_chunks", pdf_text, muumi_text, finrus_text, adapted_text, chunk_size=10000)

Сохранён файл: output_chunks/text_chunk_1.csv, слов: 9993
Сохранён файл: output_chunks/text_chunk_2.csv, слов: 9994
Сохранён файл: output_chunks/text_chunk_3.csv, слов: 9997
Сохранён файл: output_chunks/text_chunk_4.csv, слов: 10000
Сохранён файл: output_chunks/text_chunk_5.csv, слов: 9999
Сохранён файл: output_chunks/text_chunk_6.csv, слов: 9998
Сохранён файл: output_chunks/text_chunk_7.csv, слов: 9999
Сохранён файл: output_chunks/text_chunk_8.csv, слов: 9990
Сохранён файл: output_chunks/text_chunk_9.csv, слов: 9993
Сохранён файл: output_chunks/text_chunk_10.csv, слов: 10000
Сохранён файл: output_chunks/text_chunk_11.csv, слов: 10000
Сохранён файл: output_chunks/text_chunk_12.csv, слов: 9997
Сохранён файл: output_chunks/text_chunk_13.csv, слов: 9997
Сохранён файл: output_chunks/text_chunk_14.csv, слов: 9994
Сохранён файл: output_chunks/text_chunk_15.csv, слов: 9996
Сохранён файл: output_chunks/text_chunk_16.csv, слов: 9986
Сохранён файл: output_chunks/text_chunk_17.csv, слов: 10000
Со

Загружаю новостной и книжный корпуса и объединяю в единый корпус.

In [50]:
import pandas as pd

df_articles = pd.read_csv("all_articles_combined.csv")

df_articles = df_articles.loc[:, ~df_articles.columns.duplicated()]
df_articles = df_articles.rename(columns={"text_raw": "text"})
df_articles = df_articles[["text"]]
df_articles["source"] = "news"

chunk_dir = "output_chunks"
chunk_files = sorted([f for f in os.listdir(chunk_dir) if f.endswith(".csv")])

lesson_chunks = []
for file in chunk_files:
    df = pd.read_csv(os.path.join(chunk_dir, file))
    df["source"] = "book"
    lesson_chunks.append(df)

df_lessons = pd.concat(lesson_chunks, ignore_index=True)

df_combined = pd.concat([df_articles, df_lessons], ignore_index=True)

df_combined.to_csv("complete_corpus.csv", index=False, encoding="utf-8")

Счетчик слов:

In [51]:
import pandas as pd

df = pd.read_csv("complete_corpus.csv")

def count_words(text):
    return len(str(text).split())

df["word_count"] = df["text"].apply(count_words)

total_words = df["word_count"].sum()

print(f"Общее количество слов в корпусе: {total_words}")

Общее количество слов в корпусе: 449861
